# Fooling Image Classifiers

  - **Author**: Henning Heyen
  - **Date**: 05/04/2023


Welcome to this notebook!

The goal of this project is to show how state-of-the-art image classifiers behave when the
input images are **out of distribution**.

We use common Imagenet pre-trained models to pass
the images through. We then build an ensemble model and measure its prediction uncertainty.Then we actively try to fool one of the models using different perturbated images and investigate their saliency maps.

Please find an in depth **report** on this project in the [GitHub repository](https://github.com/henningheyen/FoolingImageClassifiers).

To run this notebook please use Google Colab as all libraries (especially TF Keras) are preinstalled there.

# Table of Contents

[1 Collecting a test set with epistemic uncertainty and passing it through image classifiers](#1)

[2 Building an uncertainty quantification ensemble](#2)

[3 Fooling one of the models](#3)

[4 Analysing saliency maps for the images](#4)




<a id='1'></a>
# 1 Collecting a test set with epistemic uncertainty and passing it through image classifiers

In [ ]:
#@title mounting drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title helper methods

# importing an image, change the path if necessary
def path(image_name):
  data_path = '/content/drive/MyDrive/images'
  return os.path.join(data_path, image_name)

In [ ]:
#@title Imports
# imports
import numpy as np
import pandas as pd
import os, sys
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import PIL.Image
from matplotlib import pylab as P
import os, sys

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import preprocess_input as xception_preprocess_input
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.imagenet_utils import decode_predictions

In [ ]:
# list of images
umbrellas = [load_img(path(f'umbrella{i}.jpeg'), keep_aspect_ratio=True) for i in range(0,6)]

# plotting images
f, axarr = plt.subplots(2,3, figsize=(20, 10))
axarr[0,0].imshow(umbrellas[0])
axarr[0,0].set_title('umbrella0')
axarr[0,0].axis('off')
axarr[0,1].imshow(umbrellas[1])
axarr[0,1].set_title('umbrella1')
axarr[0,1].axis('off')
axarr[0,2].imshow(umbrellas[2])
axarr[0,2].set_title('umbrella2')
axarr[0,2].axis('off')
axarr[1,0].imshow(umbrellas[3])
axarr[1,0].set_title('umbrella3')
axarr[1,0].axis('off')
axarr[1,1].imshow(umbrellas[4])
axarr[1,1].set_title('umbrella4')
axarr[1,1].axis('off')
axarr[1,2].imshow(umbrellas[5])
axarr[1,2].set_title('umbrella5')
axarr[1,2].axis('off')
plt.show()



In [ ]:
# keras pretrained models
vgg16 = VGG16(weights='imagenet')
resnet50 = ResNet50(weights='imagenet')
inceptionv3 = InceptionV3(weights='imagenet')
xception = Xception(weights='imagenet')
mobilenetv2 = MobileNetV2(weights='imagenet')

models = [
    vgg16,
    resnet50,
    inceptionv3,
    xception,
    mobilenetv2,
]

# preprocessing methods for each classifier
preprocess_methods = [
    vgg_preprocess_input,
    resnet_preprocess_input,
    inception_preprocess_input,
    xception_preprocess_input,
    mobilenet_preprocess_input,
]

In [ ]:
# Overview on models
data = {
    'model': ['vgg16', 'resnet50', 'inception_v3', 'xception', 'mobilenetv2'],
    'top 1 accuracy': [0.713, 0.749, 0.779, 0.790, 0.713],
    'top 5 accuracy': [0.901, 0.921, 0.938, 0.949, 0.901],
    'parameters': ['138.4M', '25.6M', '23.9M', '22.9M', '3.5M'],
    'depth': [16, 50, 189, 81, 105],
    'special characteristics': ['simple and uniform architecture',
                                 'residual blocks, state of the art',
                                 'multiple parallel convolutional operations',
                                 'depthwise separable convolutions',
                                 'efficient on mobile devices, fewer parameters']
}

df = pd.DataFrame(data)
df.head()

In [ ]:
def predict_image(model, preprocess, image_name, top_k=3):
  '''
    @param:
      model: pretrained classifier
      preprocess: preprocess method associated with the model
      image_name: string of the image like to be imported, see path() helper function
      top_k: k top predictions from imagenet

    @return:
      list of top k predictions with certainty
  '''

  # loading and formatting image
  input_shape = model.input_shape[1:3]
  img = load_img(path(image_name), target_size=input_shape)
  img_arr = img_to_array(img)
  img_arr = np.expand_dims(img_arr, axis=0)

  # preprocess and predict image
  processed_img = preprocess(img_arr.copy())
  preds = model.predict(processed_img, verbose=0)
  top_preds = decode_predictions(preds, top=top_k)[0]

  return top_preds


In [ ]:
all_top_preds = []

# predincting for all models on each image
for j in range(0,6):
  print('\n', '-'*50, f'umbrella{j}', '-'*50)
  for i, model in enumerate(models):
    image_name = f'umbrella{j}.jpeg'
    preprocess = preprocess_methods[i]
    top_preds = predict_image(model=model, preprocess=preprocess, image_name=image_name)
    all_top_preds.append([(top_preds[i][1], round(top_preds[i][2],3)) for i in range(len(top_preds))])
    print(f'Top 3 predictions for {model.name}: {top_preds}')


In [ ]:
# Creating a dataframe with the results

# Define the list of models and images
model_names = ["vgg16", "resnet50", "inception_v3", "xception", "mobilenetv2"]
image_names = ["umbrella0", "umbrella1", "umbrella2", "umbrella3", "umbrella4", "umbrella5"]

# Initialize empty lists for each column
model_names_list = []
image_names_list = []
prediction_list = []

# Iterate over the models and images
for i,image_name in enumerate(image_names):
  for j,model_name in enumerate(model_names):

    # Append the values to the corresponding lists
    model_names_list.append(model_name)
    image_names_list.append(image_name)
    prediction_list.append(all_top_preds[i*5+j])

# Create the dataframe
df = pd.DataFrame({
    "model": model_names_list,
    "image": image_names_list,
    "Top 3 predictions": prediction_list,
})


# showing the full entries
pd.set_option('display.max_colwidth', None)

df.head(30)

In [ ]:
# Computing all models accuracies on the 6 test images

accuracies = []
idx = [0,5,10,15,20,25]

# testing for correct prediction
for i in range(5):
  accuracies.append(sum([all_top_preds[i+j][0][0]=='umbrella' for j in idx])/6)

# creating dataframe
df_acc = pd.DataFrame({'model':model_names, 'accuracy':accuracies})
df_acc.head()



<a id='2'></a>
# 2 Building an uncertainty quantification ensemble

In [ ]:
# Using all five models and average the predictions:
def predict_avg(image_name, top_k=1):

    preds = []

    # predicting on each model and saving the results
    for model, preprocess in zip(models, preprocess_methods):
      # loading and formatting image
      input_shape = model.input_shape[1:3]
      img = image.load_img(path(image_name), target_size=input_shape)
      img_arr = image.img_to_array(img)
      img_arr = np.expand_dims(img_arr, axis=0)

      # preprocess and predict image
      processed_img = preprocess(img_arr.copy())
      pred = model.predict(processed_img, verbose=0)[0]
      preds.append(pred)

    # aggregarting the predictions by averaging
    avg_pred = np.mean(preds, axis=0).reshape(1,1000)

    #decoding the average prediction
    top_preds = decode_predictions(avg_pred, top=top_k)[0]

    return top_preds


In [ ]:
all_top_ens_preds = []

# using the ensemble predicting on each image
for j in range(0,6):
  print('\n', '-'*50, f'umbrella{j}', '-'*50)
  image_name = f'umbrella{j}.jpeg'
  top_preds = predict_avg(image_name)
  all_top_ens_preds.append([(top_preds[i][1], round(top_preds[i][2],3)) for i in range(len(top_preds))])
  print(f'Top 1 predictions for ensemble: {top_preds}')


In [ ]:
df_ensemble = pd.DataFrame({'image':image_names,
                            'prediction': [all_top_ens_preds[i][0][0] for i in range(6)],
                            'max probability': [all_top_ens_preds[i][0][1] for i in range(6)]})
df_ensemble

In [ ]:
# Extending the accuracy table by the ensemble
new_row = pd.DataFrame({'model':['ensemble'], 'accuracy':[sum([all_top_ens_preds[i][0][0]=='umbrella' for i in range(6)])/6]})
df_acc = pd.concat([df_acc, new_row], axis=0, ignore_index=True)
df_acc

<a id='3'></a>
# 3 Fooling one of the models

In [ ]:
# helper method to plot an image
def plot(image_name):
  img = load_img(path(f'{image_name}.jpeg'), keep_aspect_ratio=True)
  plt.figure(figsize=(8,6))
  plt.imshow(img)
  plt.title(image_name)
  plt.show()

In [ ]:
'''
  Creating some noise images:

 For reproducability purposes this code is commented out
 since the noise added is random and might therefore give
 different predictions.
'''


'''
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from skimage.util import random_noise


for im in ['flamingo','zebra']:

  # Load the image
  img = cv2.imread(path(f'{im}0.jpeg'))

  # Adding salt-and-pepper noise to the image.
  noise_img = random_noise(img, mode='s&p', amount=0.3)
  noise_img = np.array(255*noise_img, dtype = 'uint8')

  # save the image
  # cv2_imshow(noise_img) # uncomment to show image
  cv2.imwrite(path(f'{im}14.jpeg'), noise_img)
'''

In [ ]:
# Predicting on all zebra images and saving the results
preds_list_zebra = []
probs_list_zebra = []

for i in range(15):
  preds = predict_image(xception, xception_preprocess_input, f'zebra{i}.jpeg')
  preds_list_zebra.append(preds[0][1])
  probs_list_zebra.append(round(preds[0][2],3))
  print(100*'-', f'\n Top 3 predictions for zebra{i}: {preds}')
  plot(f'zebra{i}')



In [ ]:
# Predicting on all flamingo images and saving the results
preds_list_flamingo = []
probs_list_flamingo = []

for i in range(15):
  preds = predict_image(xception, xception_preprocess_input, f'flamingo{i}.jpeg')
  preds_list_flamingo.append(preds[0][1])
  probs_list_flamingo.append(round(preds[0][2],3))
  print(100*'-', f'\n Top 3 predictions for flamingo{i}: {preds}')
  plot(f'flamingo{i}')

In [ ]:
# table on zebra results
df_zebra = pd.DataFrame({'image': [f'zebra{i}' for i in range(15)],
                         'prediction': preds_list_zebra,
                         'probability': probs_list_zebra})

df_zebra

In [ ]:
# table on flamingo results
df_flamingo = pd.DataFrame({'image': [f'flamingo{i}' for i in range(15)],
                         'prediction': preds_list_flamingo,
                         'probability': probs_list_flamingo})

df_flamingo

In [ ]:
# Calculating accuracies on all fooling images (15 zebras, 15 flamingos)
accuracy_zebra = []
accuracy_flamingo = []

for model,preprocess in zip(models,preprocess_methods):

  preds_list_flamingo = []
  probs_list_flamingo = []

  for i in range(15):
    preds = predict_image(model, preprocess, f'flamingo{i}.jpeg')
    preds_list_flamingo.append(preds[0][1])
    probs_list_flamingo.append(round(preds[0][2],3))

  # calculating and appending the results
  accuracy_zebra.append(sum([preds_list_zebra[i]=='zebra' for i in range(15)])/15)
  accuracy_flamingo.append(round(sum([preds_list_flamingo[i]=='flamingo' for i in range(15)])/15,3))



In [ ]:
# table on accuracies for fooling images
df_acc_fool = pd.DataFrame({'model': model_names,
                            'accuracy zebra': accuracy_zebra,
                            'accuracy flamingo': accuracy_flamingo,})

df_acc_fool



<a id='4'></a>
# 4 Analysing saliency maps for the images

In [ ]:
#@title load saliency module

# Install PAIR Saliency Library
!pip install saliency
# From PAIR saliency repository.
import saliency.core as saliency


In [ ]:
#@title helper methods
def LoadImage(file_path):
  im = PIL.Image.open(file_path)
  im = im.resize((299,299))
  im = np.asarray(im)
  return im


In [ ]:
# saving all image names in one list
umbrella_names = [f'umbrella{i}' for i in range(6)]
zebra_names = [f'zebra{i}' for i in range(15)]
flamingo_names = [f'flamingo{i}' for i in range(15)]

all_image_names = [*umbrella_names, *zebra_names, *flamingo_names]

In [ ]:
# From Lab 4, call_model_function to create saliency maps, model was replaced by model_xception
class_idx_str = 'class_idx_str'
def call_model_function(images, call_model_args=None, expected_keys=None):
    target_class_idx =  call_model_args[class_idx_str]
    images = tf.convert_to_tensor(images)
    with tf.GradientTape() as tape:
        if expected_keys==[saliency.base.INPUT_OUTPUT_GRADIENTS]:
            tape.watch(images)
            output_layer = model_xception(images)
            output_layer = output_layer[:,target_class_idx]
            gradients = np.array(tape.gradient(output_layer, images))
            return {saliency.base.INPUT_OUTPUT_GRADIENTS: gradients}
        else:
            conv_layer, output_layer = model_xception(images)
            gradients = np.array(tape.gradient(output_layer, conv_layer))
            return {saliency.base.CONVOLUTION_LAYER_VALUES: conv_layer,
                    saliency.base.CONVOLUTION_OUTPUT_GRADIENTS: gradients}

In [ ]:
# importing the imagenet labels, change path if necessary
import json

# Change path if necessary
with open('/content/drive/MyDrive/imagenet_class_index.json', 'r') as f:
    class_idx = json.load(f)

In [ ]:
# Similar to Lab 4 creating a keras model based on the pretrained xception
model_xception = tf.keras.models.Model([xception.inputs], [xception.output])

In [ ]:
# Creating SmoothGrad Saliency maps for all images using the xception model
for image_name in all_image_names:
  # Load the image
  im_orig = LoadImage(path(f'{image_name}.jpeg'))

  # remove potential 4th channel to retrieve RBG format
  if im_orig.shape[-1] > 3:
    im_orig = im_orig[:, :, :3]

  #preprocess image
  im = xception_preprocess_input(im_orig)

  # predict image using the moethod from task 1
  preds = predict_image(xception, xception_preprocess_input, image_name=f'{image_name}.jpeg')
  # extracting the class index
  prediction_class = int(list(class_idx.keys())[list(class_idx.values()).index(list(preds[0][0:2]))])
  # create argument for GetSmoothedMask
  call_model_args = {class_idx_str: prediction_class}

  #print prediction and max probability
  print(100*'-', f"\n Prediction for {image_name}: ", preds[0][1:3])

  # Construct the saliency object
  gradient_saliency = saliency.GradientSaliency()

  # Compute SmoothGrad mask.
  smoothgrad_mask_3d = gradient_saliency.GetSmoothedMask(im, call_model_function, call_model_args)

  # Call the visualization methods to convert the 3D tensors to 2D grayscale.
  smoothgrad_mask_grayscale = saliency.VisualizeImageGrayscale(smoothgrad_mask_3d)

  # Figure
  f = plt.figure(figsize=(10,5))
  f.add_subplot(1,2, 1)

  P.axis('off')
  P.imshow(im_orig)
  P.title(image_name)
  f.add_subplot(1,2, 2)

  P.axis('off')
  P.imshow(smoothgrad_mask_grayscale, cmap=P.cm.gray, vmin=0, vmax=1)
  P.title(f'SmoothGrad {image_name}')
  plt.subplots_adjust(wspace=0, hspace=0)
  P.show()